In [1]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_API_KEY"),  
    api_version="2025-03-01-preview",
    azure_endpoint = "https://aisg-sj.openai.azure.com/" # o4-mini
    # azure_endpoint = "https://decla-mbncunfi-australiaeast.cognitiveservices.azure.com/" # o3-mini
    )

# Upload a file with a purpose of "batch"
file = client.files.create(
  file=open("example_batch_input.jsonl", "rb"), 
  purpose="batch",
  extra_body={"expires_after":{"seconds": 1209600, "anchor": "created_at"}} # Optional you can set to a number between 1209600-2592000. This is equivalent to 14-30 days
)


print(file.model_dump_json(indent=2))

{
  "id": "file-3b6e5721675449a98743b06d270c996b",
  "bytes": 1625,
  "created_at": 1749631427,
  "filename": "example_batch_input.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": 1750841027,
  "status_details": null
}


In [2]:
import datetime

print(f"File expiration: {datetime.datetime.fromtimestamp(file.expires_at) if file.expires_at is not None else 'Not set'}")

file_id = file.id

File expiration: 2025-06-25 16:43:47


In [3]:
# Submit a batch job with the file
batch_response = client.batches.create(
    input_file_id=file_id,
    endpoint="/chat/completions",
    completion_window="24h",
    extra_body={"output_expires_after":{"seconds": 1209600, "anchor": "created_at"}} # Optional you can set to a number between 1209600-2592000. This is equivalent to 14-30 days
)


# Save batch ID for later use
batch_id = batch_response.id

print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14",
  "completion_window": "24h",
  "created_at": 1749631437,
  "endpoint": "/chat/completions",
  "input_file_id": "file-3b6e5721675449a98743b06d270c996b",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1749717834,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


In [4]:
import time
import datetime 

status = "validating"
while status not in ("completed", "failed", "canceled"):
    time.sleep(60)
    batch_response = client.batches.retrieve(batch_id)
    status = batch_response.status
    print(f"{datetime.datetime.now()} Batch Id: {batch_id},  Status: {status}")

if batch_response.status == "failed":
    for error in batch_response.errors.data:  
        print(f"Error code {error.code} Message {error.message}")

2025-06-11 16:44:58.510814 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: validating
2025-06-11 16:45:59.458582 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: validating
2025-06-11 16:47:00.425928 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: in_progress
2025-06-11 16:48:01.360222 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: in_progress
2025-06-11 16:49:02.342012 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: in_progress
2025-06-11 16:50:03.274232 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: in_progress
2025-06-11 16:51:04.211146 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: in_progress
2025-06-11 16:52:05.197377 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: finalizing
2025-06-11 16:53:06.181496 Batch Id: batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14,  Status: completed


In [9]:
import json

output_file_id = batch_response.output_file_id

if not output_file_id:
    output_file_id = batch_response.error_file_id

verification_results = []
error_sample_ids = []

if output_file_id:
    file_response = client.files.content(output_file_id)
    raw_responses = file_response.text.strip().split('\n')  

    for raw_response in raw_responses:  
        json_response = json.loads(raw_response)  
        
        # Check for error status codes
        if json_response["response"]["status_code"] != 200:
            error_sample_ids.append(json_response["custom_id"])
            continue
        
        # Create new JSON format with desired schema for successful responses
        verification_entry = {
            "custom_id": json_response["custom_id"],
            "verification_response": json_response["response"]["body"]["choices"][0]["message"]["content"]
        }
        
        verification_results.append(verification_entry)
        
        # Optional: print original for debugging
        formatted_json = json.dumps(json_response, indent=2)  
        print(formatted_json)

# Save verification results to file
with open('verification_results.json', 'w') as f:
    json.dump(verification_results, f, indent=2)

print(f"\nSaved {len(verification_results)} verification results to verification_results.json")

if error_sample_ids:
    print(f"Error sample IDs with status_code != 200: {error_sample_ids}")
else:
    print("All samples processed successfully with status_code 200")

{
  "custom_id": "request-1",
  "response": {
    "body": {
      "choices": [
        {
          "content_filter_results": {
            "hate": {
              "filtered": false,
              "severity": "safe"
            },
            "self_harm": {
              "filtered": false,
              "severity": "safe"
            },
            "sexual": {
              "filtered": false,
              "severity": "safe"
            },
            "violence": {
              "filtered": false,
              "severity": "safe"
            }
          },
          "finish_reason": "stop",
          "index": 0,
          "logprobs": null,
          "message": {
            "annotations": [],
            "content": "It\u2019s the \u201csmiling face with heart-eyes\u201d emoji. A round, bright yellow face with a big, enthusiastic grin and two red hearts replacing its eyes\u2014used to show love, adoration, or delight.",
            "refusal": null,
            "role": "assistant"
       

In [10]:
print(client.batches.list().model_dump_json(indent=2))

{
  "data": [
    {
      "id": "batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14",
      "completion_window": "24h",
      "created_at": 1749631437,
      "endpoint": "/chat/completions",
      "input_file_id": "file-3b6e5721675449a98743b06d270c996b",
      "object": "batch",
      "status": "completed",
      "cancelled_at": null,
      "cancelling_at": null,
      "completed_at": 1749631958,
      "error_file_id": null,
      "errors": null,
      "expired_at": null,
      "expires_at": 1749717834,
      "failed_at": null,
      "finalizing_at": 1749631887,
      "in_progress_at": 1749631640,
      "metadata": null,
      "output_file_id": "file-da79c320-ee92-40f9-9167-86b270259bc9",
      "request_counts": {
        "completed": 1,
        "failed": 0,
        "total": 1
      }
    },
    {
      "id": "batch_8d9364c9-7bba-470e-b80d-a6278e99a62e",
      "completion_window": "24h",
      "created_at": 1749622944,
      "endpoint": "/chat/completions",
      "input_file_id": "file-7ba74b8

In [6]:
all_jobs = []
# Automatically fetches more pages as needed.
for job in client.batches.list(
    limit=20,
):
    # Do something with job here
    all_jobs.append(job)
print(all_jobs)

[Batch(id='batch_ade6952d-64bb-4d1d-ab4c-57b1dbdc9d14', completion_window='24h', created_at=1749631437, endpoint='/chat/completions', input_file_id='file-3b6e5721675449a98743b06d270c996b', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1749631958, error_file_id=None, errors=None, expired_at=None, expires_at=1749717834, failed_at=None, finalizing_at=1749631887, in_progress_at=1749631640, metadata=None, output_file_id='file-da79c320-ee92-40f9-9167-86b270259bc9', request_counts=BatchRequestCounts(completed=1, failed=0, total=1)), Batch(id='batch_8d9364c9-7bba-470e-b80d-a6278e99a62e', completion_window='24h', created_at=1749622944, endpoint='/chat/completions', input_file_id='file-7ba74b86dcb14621be65d4095c4ed1ce', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1749623742, error_file_id=None, errors=None, expired_at=None, expires_at=1749709341, failed_at=None, finalizing_at=1749623676, in_progress_at=17496231

In [ ]:
import time
from openai import BadRequestError

max_retries = 10
retries = 0
initial_delay = 5
delay = initial_delay

while True:
    try:
        batch_response = client.batches.create(
            input_file_id=file_id,
            endpoint="/chat/completions",
            completion_window="24h",
        )
        
        # Save batch ID for later use
        batch_id = batch_response.id
        
        print(f"✅ Batch created successfully after {retries} retries")
        print(batch_response.model_dump_json(indent=2))
        break  
        
    except BadRequestError as e:
        error_message = str(e)
        
        # Check if it's a token limit error
        if 'token_limit_exceeded' in error_message:
            retries += 1
            if retries >= max_retries:
                print(f"❌ Maximum retries ({max_retries}) reached. Giving up.")
                raise
            
            print(f"⏳ Token limit exceeded. Waiting {delay} seconds before retry {retries}/{max_retries}...")
            time.sleep(delay)
            
            # Exponential backoff - increase delay for next attempt
            delay *= 2
        else:
            # If it's a different error, raise it immediately
            print(f"❌ Encountered non-token limit error: {error_message}")
            raise